In [1]:
import pandas as pd
import numpy as np
import glob

#### BEAM 

In [1]:
#File location S3
locSF_rh = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2010-lessrh-20220223/beam/year-2010-iteration-2/ITERS/it.0/"
loc_2018_tripid_modified = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220327/beam/year-2018-iteration-5/ITERS/it.0/"

In [3]:
%%time
#Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "reason": "category",
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object"
}

# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(loc_2018_tripid_modified + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:21: DtypeWarning: Columns (5,7,12,13,14,15,24,30,33,50,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 3min 37s


In [4]:
#Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
#Rename mode column
eventsSF.rename(columns={"mode":"modeType"}, inplace=True)  

In [6]:
#Replace "Work" with "work" in actType column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [8]:
eventsSF.tail()

,person,vehicle,time,type,modeType,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,price,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,endY,endX,startY,startX,arrivalTime,departureTime,secondaryFuelLevel,primaryFuelLevel,cost,driver,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,links,numPassengers,primaryFuel,departTime,score,incentive,tollCost,netCost
36392007,5083597,213765,215783.0,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36392008,NaN,213765,215783.0,LeavingParkingEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,666.0,None,FlatFee,Workplace,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5083597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0,NaN,NaN,NaN
36392009,556788,NaN,215915.0,ModeChoice,hov3_teleportation,hov3_teleportation,NaN,HOV3_TELEPORTATION,46346.0,true,7671.847,2.0,"WALK,CAR_HOV3,WALK","body-556788,teleportationSharedVehicle-2-55678...",escort,shopping,182626730.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36392010,556788,NaN,215915.0,actend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46346.0,NaN,NaN,escort,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36392011,556788,NaN,215915.0,departure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46346.0,hov3_teleportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
eventsSF[eventsSF['person']=='556788'] 182626713 182626717

,person,vehicle,time,type,modeType,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,price,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,endY,endX,startY,startX,arrivalTime,departureTime,secondaryFuelLevel,primaryFuelLevel,cost,driver,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,links,numPassengers,primaryFuel,departTime,score,incentive,tollCost,netCost
27143983,556788,NaN,64911.0,ModeChoice,walk,walk,NaN,WALK,42966.0,false,103386.901880,1.0,WALK,body-556788,Home,school,182626713.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27148245,556788,NaN,64911.0,actend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42966.0,NaN,NaN,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27148246,556788,NaN,64911.0,departure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42966.0,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27148249,556788,body-556788,64911.0,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36380608,556788,body-556788,139829.0,PersonLeavesVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36380611,556788,NaN,139829.0,arrival,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46306.0,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36380614,556788,NaN,139829.0,actstart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46306.0,NaN,NaN,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36380758,556788,NaN,139979.0,ModeChoice,walk,walk,NaN,WALK,46306.0,false,50316.194302,1.0,WALK,body-556788,school,eatout,182626717.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36380759,556788,NaN,139979.0,actend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46306.0,NaN,NaN,school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36380760,556788,NaN,139979.0,departure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46306.0,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person']
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver'])
                     , eventsSF['person'], np.nan)
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person'])
                     , eventsSF['driver'], np.nan)

In [8]:
#Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [9]:
#Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [10]:
%%time
#Split the riders column
dfRiders = eventsSF['riders'].str.split(':',expand=True)

Wall time: 5min 51s


In [11]:
riderStartIndex = len(eventsSF.columns)

In [12]:
#Concat the new riders columns to the main dataframe
eventsSF = pd.concat([eventsSF, dfRiders.reindex(eventsSF.index)], axis=1)

In [13]:
%%time
# Making a riderID column and merge it with the already merged id column 
NaN = np.nan
eventsSF['ID'] = NaN 
    
for col in eventsSF.columns[riderStartIndex:-2]:
    eventsSF['riderID'] = np.where(eventsSF[col].isin(eventsSF["IDMerged"]), eventsSF[col], np.nan)
    eventsSF['ID'] = eventsSF['ID'].combine_first(eventsSF['riderID'])

Wall time: 5min 51s


In [14]:
eventsSF['IDMerged'] = eventsSF['IDMerged'].combine_first(eventsSF['ID'])

In [15]:
#dropping unused riders columns
eventsSF = eventsSF.drop(columns=eventsSF.columns[riderStartIndex:])

In [96]:
# shift column 'Name' to first position
first_column = eventsSF.pop('IDMerged')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [98]:
%%time
#To select just the numeric MergedID
#eventsSF.loc[eventsSF.IDMerged.str.isnumeric()]
eventsSFnum = eventsSF[pd.to_numeric(eventsSF.IDMerged, errors='coerce').notnull()].sort_values(['IDMerged','time']).reset_index(drop=True)

Wall time: 2min 1s


In [76]:
eventsSFnum['actEndTime'] = np.where(eventsSFnum['type']=='actend'
                     , eventsSFnum['time'], np.nan)  

In [77]:
eventsSFnum['actStartTime'] = np.where(eventsSFnum['type']=='actstart'
                     , eventsSFnum['time'], np.nan)  

In [78]:
eventsSFnum['PTduration'] = np.where(eventsSFnum['type']=='PathTraversal'
                     , eventsSFnum['arrivalTime'] - eventsSFnum['departureTime'], np.nan)  

In [79]:
eventsSFnum['duration_walking'] = np.where(eventsSFnum['modeType']=='walk'
                     , eventsSFnum['PTduration'], np.nan)

In [80]:
eventsSFnum['duration_in_privateCar'] = np.where((eventsSFnum['modeType']=='car')|(eventsSFnum['modeType']=='car_hov3')|(eventsSFnum['modeType']=='car_hov2')|
                                              (eventsSFnum['modeType']=='hov2_teleportation')|(eventsSFnum['modeType']=='hov3_teleportation')
                                                 , eventsSFnum['PTduration'], np.nan)

In [81]:
eventsSFnum['duration_on_bike'] = np.where(eventsSFnum['modeType']=='bike', eventsSFnum['PTduration'], np.nan)

In [82]:
eventsSFnum['duration_in_ridehail'] = np.where((eventsSFnum['modeType']=='ride_hail')|(eventsSFnum['modeType']=='ride_hail_pooled'), eventsSFnum['PTduration'], np.nan)

In [83]:
eventsSFnum['duration_in_public'] = np.where((eventsSFnum['modeType']=='bike_transit')|(eventsSFnum['modeType']=='drive_transit')|(eventsSFnum['modeType']=='walk_transit'), eventsSFnum['PTduration'], np.nan)

In [84]:
eventsSFnum["tripIndex"] = eventsSFnum.groupby("IDMerged")["tourIndex"].rank(method="first", ascending=True)
eventsSFnum["tripIndex"] = eventsSFnum.tripIndex.fillna(method='ffill')

In [85]:
eventsSFnum["currentTourModeFillna"] = eventsSFnum.currentTourMode.fillna(method='ffill')#.astype('object')

In [86]:
#Change column type to numeric to be able to sort_values
eventsSFnum["IDMerged"] = pd.to_numeric(eventsSFnum["IDMerged"])

In [87]:
eventsSFnum['actEndType'] = np.where(eventsSFnum['type']=='actend'
                     , eventsSFnum['actType'], "")

In [88]:
eventsSFnum['actStartType'] = np.where(eventsSFnum['type']=='actstart'
                     , eventsSFnum['actType'], "")

In [91]:
eventsSFtrips = pd.pivot_table(
   eventsSFnum,
   index=['IDMerged','tripIndex','currentTourModeFillna'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'PTduration': np.sum, 'primaryFuel': np.sum, 
            'netCost': np.sum, 'actStartType': np.sum, 'actEndType': np.sum, 'duration_walking': np.sum,
            'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 'duration_in_ridehail': np.sum, 
            'duration_in_public': np.sum}).reset_index()

In [92]:
#Adding door to door time
eventsSFtrips['DDtime'] = eventsSFtrips['actStartTime'] - eventsSFtrips['actEndTime'] 

In [93]:
eventsSFtrips['actPurpose'] = eventsSFtrips['actEndType'] + "_to_" + eventsSFtrips['actStartType']

In [94]:
#Rename the column
eventsSFtrips = eventsSFtrips.rename(columns={'currentTourModeFillna': 'currentTourMode'})

In [95]:
eventsSFtrips.head()

,IDMerged,tripIndex,currentTourMode,PTduration,actEndTime,actEndType,actStartTime,actStartType,duration_in_privateCar,duration_in_public,duration_in_ridehail,duration_on_bike,duration_walking,netCost,primaryFuel,DDtime,actPurpose
0,4,1.0,car,477.0,54694.0,Home,55180.0,othdiscr,477.0,0.0,0.0,0.0,0.0,1.258902,1.717840e+07,486.0,Home_to_othdiscr
1,4,2.0,car,0.0,0.0,,60702.0,Home,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,60702.0,_to_Home
2,8,1.0,car,11817.0,71933.0,shopping,83781.0,Home,0.0,0.0,0.0,0.0,11817.0,0.052631,0.000000e+00,11848.0,shopping_to_Home
3,8,2.0,car,11510.0,60454.0,Home,71933.0,shopping,31.0,0.0,0.0,0.0,11479.0,0.000000,6.498952e+05,11479.0,Home_to_shopping
4,66,1.0,car,102.0,0.0,,0.0,,0.0,0.0,0.0,0.0,102.0,1.210568,6.998915e+03,0.0,_to_


#### Activity Sim

In [9]:
loc = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2018-base-20220327/activitysim/" 
trips = pd.read_csv(loc + 'final_trips.csv')
households = pd.read_csv(loc + 'final_households.csv')
persons = pd.read_csv(loc + 'final_persons.csv')
tours = pd.read_csv(loc +'final_tours.csv')
plans = pd.read_csv(loc +'final_plans.csv')
landuse = pd.read_csv(loc +'final_land_use.csv')
#checkpoints = pd.read_csv(loc + 'final_checkpoints.csv')
#joint_tour_participants = pd.read_csv(loc + 'final_joint_tour_participants.csv')
#specs = pd.read_csv(loc +'final_trip_mode_choice_specs.csv')
#tripModeChoice = pd.read_csv(loc +'final_trip_mode_choice_raw.csv')
#tripModeChoiceUtilities = pd.read_csv(loc +'final_trip_mode_choice_utilities.csv')

In [16]:
#Merge households and persons 
persons = persons.sort_values(by=['household_id'])
households = households.sort_values(by=['household_id'])
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [17]:
#Merge tours, households and persons
tours = tours.sort_values(by=['person_id'])
hhpersons = hhpersons.sort_values(by=['person_id'])
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [18]:
%%time
#Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id'])
hhperTours = hhperTours.sort_values(by=['person_id','tour_id'])
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

Wall time: 10.1 s


In [22]:
tourTripsMerged.tail()

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,tdd,start,end,duration,composition,tour_mode,atwork_subtour_frequency,parent_tour_id,stop_frequency,earning,worker,student,hispanic.1,person_sex,PNUM,sex,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
2530171,2427031157,7399485,2894969,303378894,escort,1,False,2,escort,451,426,16.878273,7.0,SHARED3PAY,1.142043,escort,1,1,1,1,non_mandatory,1,37.0,7.0,7.0,0.0,NaN,SHARED3FREE,NaN,NaN,0out_1in,3000.0,0,0,0,male,3,1,35.0,5,20 to 35,16.0,-1,1,white,0,23,-1,no,2,0,445,4,3,3,-121.999243,37.359064,False,True,True,True,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,0,False,445,10.246107,-1,NaN,0.0,-1,NaN,NaN,False,NaN,0.0,0.0,0.0,False,N,True,False,False,False,NaN,False,False,0,0,1,32,1,1,0,0,0,0,0,0,0,0.0,2,6085,yes,60855085072009,1,no,41,1,rent not recent,four or more,no,2,two or more,45000.0,gt35-lt65,2010001310391,1.0,white,gt30-lt60,0,one,0,no,4,7,4,0.097,271926,45.0,2,8.81,10.246107,3.0,3,3,1,0,1,0,1,0,True,False,True,False,2,0.312611,0,4,3,0,1,4,True,1_Main,1
2530172,2427031158,7399485,2894969,303378894,escort,2,False,2,Home,445,451,NaN,7.0,SHARED3FREE,0.981849,escort,1,1,1,1,non_mandatory,1,37.0,7.0,7.0,0.0,NaN,SHARED3FREE,NaN,NaN,0out_1in,3000.0,0,0,0,male,3,1,35.0,5,20 to 35,16.0,-1,1,white,0,23,-1,no,2,0,445,4,3,3,-121.999243,37.359064,False,True,True,True,False,True,False,False,False,True,True,False,False,False,False,False,False,False,False,0,False,445,10.246107,-1,NaN,0.0,-1,NaN,NaN,False,NaN,0.0,0.0,0.0,False,N,True,False,False,False,NaN,False,False,0,0,1,32,1,1,0,0,0,0,0,0,0,0.0,2,6085,yes,60855085072009,1,no,41,1,rent not recent,four or more,no,2,two or more,45000.0,gt35-lt65,2010001310391,1.0,white,gt30-lt60,0,one,0,no,4,7,4,0.097,271926,45.0,2,8.81,10.246107,3.0,3,3,1,0,1,0,1,0,True,False,True,False,2,0.312611,0,4,3,0,1,4,True,1_Main,1
2530173,2427063449,7399583,2895003,3033829

In [20]:
tourTripsMerged[tourTripsMerged['person_id'] == 556788]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,tdd,start,end,duration,composition,tour_mode,atwork_subtour_frequency,parent_tour_id,stop_frequency,earning,worker,student,hispanic.1,person_sex,PNUM,sex,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
189942,182626513,556788,1973984,22828314,eatout,1,True,1,eatout,500,497,NaN,18.0,SHARED2PAY,0.627566,eatout,1,1,1,1,non_mandatory,1,171.0,18.0,20.0,2.0,NaN,SHARED2PAY,NaN,NaN,0out_0in,0.0,0,1,0,male,4,1,0.0,5,19 and under,12.0,1381,6,asian,0,14,-1,no,2,0,497,7,4,1,-121.996665,37.255891,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,True,True,False,False,1,False,497,5.978868,1246,0.002042,147.68333,-1,NaN,NaN,False,NaN,0.0,0.0,0.0,False,M,True,False,False,False,school1,False,False,1,0,0,4,1,0,1,0,0,0,0,1,0,0.0,2,6085,yes,60855073015009,1,no,54,6,own not recent,four or more,yes,1,two or more,205000.0,gt35-lt65,2012001175183,2.0,asian,gt150,0,two or more,0,no,4,3,4,0.097,33634,205.0,4,12.86,8.96382,2.0,3,2,2,0,1,1,0,0,True,False,False,False,2,1.125341,0,3,1,0,2,3,True,0_tours,0
189943,182626517,556788,1973984,22828314,eatout,1,False,1,Home,497,500,NaN,20.0,SHARED2FREE,0.668390,eatout,1,1,1,1,non_mandatory,1,171.0,18.0,20.0,2.0,NaN,SHARED2PAY,NaN,NaN,0out_0in,0.0,0,1,0,male,4,1,0.0,5,19 and under,12.0,1381,6,asian,0,14,-1,no,2,0,497,7,4,1,-121.996665,37.255891,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,True,True,False,False,1,False,497,5.978868,1246,0.002042,147.68333,-1,NaN,NaN,False,NaN,0.0,0.0,0.0,False,M,True,False,False,False,school1,False,False,1,0,0,4,1,0,1,0,0,0,0,1,0,0.0,2,6085,yes,60855073015009,1,no,54,6,own not recent,four or more,yes,1,two or more,205000.0,gt35-lt65,2012001175183,2.0,asian,gt150,0,two or more,0,no,4,3,4,0.097,33634,205.0,4,12.86,8.96382,2.0,3,2,2,0,1,1,0,0,True,False,False,False,2,1.125341,0,3,1,0,2,3,True,0_tours,0
189

In [122]:
%%time
#Merge trips, tours, households and persons, and plans
plans = plans.sort_values(by=['person_id', 'trip_id']).loc[plans['ActivityElement'] == 'leg']
tourTripsMerged = tourTripsMerged.sort_values(by=['person_id','trip_id'])
plansTripsMerged = pd.merge(left=plans, right=tourTripsMerged, how='left', on=['person_id','trip_id'], suffixes=('', '_drop'))
plansTripsMerged.drop([col for col in plansTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

Wall time: 53.1 s


In [123]:
plansTripsMerged['PlanElementIndex'] = plansTripsMerged['PlanElementIndex']/2

In [114]:
#Concat mode_choice_raw files
path = "C:/Users/nazanin/Downloads/trip_mode_choice/trip_mode_choice/"
all_files = glob.glob(path + "*raw.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_raw = pd.concat(li2, axis = 0, ignore_index=True)

In [36]:
#Concat mode_choice_utilities files
path = "C:/Users/nazanin/Downloads/trip_mode_choice/trip_mode_choice/"
all_files = glob.glob(path + "*utilities.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_utilities = pd.concat(li2, axis = 0, ignore_index=True)

In [37]:
#Merge mode_choice_raw and utilities
SFmode_choice_raw = SFmode_choice_raw.sort_values(by=['trip_id'])
SFmode_choice_utilities = SFmode_choice_utilities.sort_values(by=['trip_id'])
rawUtil = pd.merge(left=SFmode_choice_raw, right=SFmode_choice_utilities, how='left', on='trip_id', suffixes=('', '_drop'))
rawUtil.drop([col for col in rawUtil.columns if 'drop' in col], axis=1, inplace=True)

In [125]:
%%time
#Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
plansTripsMerged = plansTripsMerged.sort_values(by=['trip_id'])
rawUtil = rawUtil.sort_values(by=['trip_id'])
SFActMerged= pd.merge(left=plansTripsMerged, right=rawUtil, how='left', on=['trip_id'], suffixes=('', '_drop'))
SFActMerged.drop([col for col in SFActMerged.columns if 'drop' in col], axis=1, inplace=True)

Wall time: 1min 26s


In [127]:
SFActMerged_chunk = SFActMerged[0:1000]
SFActMerged_chunk.to_csv('C:/Shared-Work/Data/CleanData/SFActMerged_chunk.csv', index = False)

#### Merging

In [126]:
eventsSFtrips_chunk = pd.read_csv("C:/Shared-Work/Data/CleanData/Chunks/eventsSFtrips_chunk.csv")   

In [134]:
%%time
eventsSFtrips_chunk = eventsSFtrips_chunk.sort_values(by=['IDMerged'])
SFActMerged_chunk = SFActMerged_chunk.sort_values(by=['person_id'])
SFBeamActMerged_chunk = pd.merge(eventsSFtrips_chunk, SFActMerged_chunk, how='left', left_on = ["IDMerged", 'tripIndex'] , right_on=['person_id', 'PlanElementIndex'], suffixes=('', '_drop'))
SFBeamActMerged_chunk.drop([col for col in SFBeamActMerged_chunk.columns if 'drop' in col], axis=1, inplace=True)

Wall time: 88.8 ms


In [136]:
SFBeamActMerged_chunk.to_csv('C:/Shared-Work/Data/CleanData/SFBeamActMerged_chunk.csv', index = False)

In [67]:
%%time
eventsSFtrips = eventsSFtrips.sort_values(by=['IDMerged'])
SFActMerged = SFActMerged.sort_values(by=['person_id'])
SFBeamActMerged = pd.merge(eventsSFtrips, SFActMerged, how='left', left_on = "IDMerged" , right_on='person_id', suffixes=('', '_drop'))
SFBeamActMerged.drop([col for col in SFBeamActMerged.columns if 'drop' in col], axis=1, inplace=True)

Wall time: 15min 55s
